In [16]:
# Importing libraries
import spotipy
import json
import webbrowser
import pandas as pd

In [17]:
import urllib.request
import spotipy.util as util

In [18]:
# Importing geolocation library and getting user's location
import geocoder
userGeolocation = geocoder.ip('me')
#print(userGeolocation.latlng)

In [19]:
# open file with keys and set the path to your credentials JSON file
credentials = "SpotifyTokens.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)
    
# read the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-currently-playing'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [20]:
#Adding traffic key

trafficKey = api_tokens["trafficKey"]

# Getting traffic data from London
request = urllib.request.Request("http://dev.virtualearth.net/REST/v1/Traffic/Incidents/51,%20-0.489,%2051.686,%200.236?key=" + trafficKey)
response = urllib.request.urlopen(request)

In [21]:
fullTrafficData = json.loads(response.read())

In [22]:
sp = spotipy.Spotify(auth=token)

In [23]:
# Getting all traffic data
trafficData = fullTrafficData["resourceSets"][0]['resources']

In [24]:
congestionDescription = []
for x in trafficData:
    if x['type'] == 5:
        congestionDescription += x['description'].split(".")
#print(congestionDescription)

In [25]:
#Creating a list to store coordinates of incidents that are congestion types and the most severe
severeCongestionCoordinates = []
for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
        severeCongestionCoordinates.append(x['point']['coordinates'])
#print(severeCongestionCoordinates)

In [26]:
# Creating a playlist for each of the traffic jams and naming it with their coordinates

#len(severeCongestionCoordinates)
for x in range(2):
    playlistCoordinateString = str(severeCongestionCoordinates[x])
    
    my_playlists = sp.user_playlist_create(user=username, name=playlistCoordinateString, public=True,
                                      description=congestionDescription[x])
    
#print(my_playlists)

In [27]:
#Hardcoding a coordinate that is in traffic
testyUserInTraffic = [51.525313, -0.260287]

hardcodedUserStatusInTraffic = False
myRealStatusInTraffic = False

# for in range loop with a hardcoded lat and long to equate to true
# probs need to hardcode the value again in the future if want to test again since live data might change

    
for x in range(len(severeCongestionCoordinates)):
    if testyUserInTraffic == severeCongestionCoordinates[x]:
        hardcodedUserStatusInTraffic = True

print(hardcodedUserStatusInTraffic)


# for in range loop with my real geolocation
for x in range(len(severeCongestionCoordinates)):
    if userGeolocation.latlng == severeCongestionCoordinates[x]:
        myRealStatusInTraffic = True

print(myRealStatusInTraffic)

True
False


In [29]:
# Making a boolean to check if song is being played or not
playingMusic = False

my_current_track = sp.current_user_playing_track()
#print(current_track['item']['name'])

# my_current_track only seems to be of type dict when a song is being played
# So if its a dictionary, music is being played
if type(my_current_track) is dict:
    playingMusic = True

# If no song is being played, print "no song being played", otherwise print the artits name and song being played
if playingMusic == False:
    print ("No song being played")
else:
    print(f"Listening to {my_current_track['item']['name']} by {my_current_track['item']['album']['artists'][0]['name']}")


Listening to tears in the club (feat. the weeknd) by FKA twigs


In [30]:
# IMPORTANT NOTE: playlist_add_items expects a lis of uris, not just one, so its needed to include the one track in a list
# read more here: https://github.com/spotipy-dev/spotipy/issues/771

if hardcodedUserStatusInTraffic == True and playingMusic == True:
    print("In traffic and playing music")
    
    
    addingCurrentSong = sp.playlist_add_items(my_playlists['id'], [my_current_track['item']['uri']], position=0)

    print(addingCurrentSong)
my_playlists['id']

In traffic and playing music
{'snapshot_id': 'Miw3ZDkyMjA0MDI5YmE3NjQzNjA0MTcxNmJhZmI1YzRkNTM5NjRkY2Yy'}


'6H4tmL52WINwGtNTnLB2n3'

In [31]:
# sp.start_playback(deviceID, None, track_selection_list)
webbrowser.open(my_playlists['external_urls']['spotify'])

True